In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re

In [4]:
data=pd.read_excel(r"C:/Users/Dell/Pictures/ccc.xlsx")
#removing all punctuations
data['content'] = data['content'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
data.head()

In [ ]:
#Sentiment part

from nltk.sentiment.vader import SentimentIntensityAnalyzer 
#nltk vader lexicon 
final_sentimentt=[]
for i in data['content'].values:
    try:
        senti=SentimentIntensityAnalyzer()
        analysis=senti.polarity_scores(i)
        final_sentimentt.append(analysis['compound'])
    except:
        final_sentimentt.append(0)
data['final_sentimentt']=final_sentimentt

pos=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt>0]
neg=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt<0]
hpos=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt>0.75]
hneg=data[['row_index','content','final_sentimentt','rating']][data.final_sentimentt<-0.25]

#Converting the polarity values from continuous to categorical
data['final_sentimentt'][data.final_sentimentt==0]= 0
data['final_sentimentt'][data.final_sentimentt > 0]= 1
data['final_sentimentt'][data.final_sentimentt< 0]= -1

In [ ]:
#changing to needed format(senti)
value = {1.0:'pos',-1.0:'neg',0.0:'neu'} 
data.final_sentimentt = [value[item] for item in data.final_sentimentt]  

In [ ]:
data.to_excel('C:/Users/Dell/Pictures/ckcheck.xlsx')

In [ ]:
#changing to needed format(for last check verification to get exact results(for avoiding type1 error))
value = {1:'1.0 out of 5 stars',2:'2.0 out of 5 stars',3:'3.0 out of 5 stars',4:'4.0 out of 5 stars',5:'5.0 out of 5 stars'} 
data.rating = [value[item] for item in data.rating]

In [ ]:
#last check
places=['1.0 out of 5 stars','2.0 out of 5 stars']
#xtracting loc and location 
pattern='|'.join(places)
x=data['rating'].str.contains(pattern)
data['final_sentimentt']=x.replace((True,False),('neg',data['final_sentimentt']))
data= data[pd.notnull(data['content'])] 
data.head()

In [7]:
#Aspect extraction part
!pip install rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
from rake_nltk import Rake
from nltk.corpus import stopwords 
from rake_nltk import Rake
from nltk.corpus import stopwords 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
#extracting aspect with higher ranking phrases (top5 min)
aspect_adj_string =[]
for i in data['content'].values:
    try:
        r= Rake()
        r = Rake(min_length=1, max_length=2)
        a=r.extract_keywords_from_text(i)
        b=r.get_ranked_phrases()[:5]
        aspect_adj_string.append(b)
    except:
        aspect_adj_string.append(0)
data['aspect_adj_string']=aspect_adj_string


In [9]:
data.to_excel("C:/Users/Dell/Downloads/ccc.xlsx")

In [ ]:
#check sample aspects
[x for x in data.aspect_adj_string.sample(10)]

In [2]:
#Topic modelling
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords  
import nltk
import re
from pywsd.utils import lemmatize_sentence
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

Warming up PyWSD (takes ~10 secs)... took 5.789640426635742 secs.


In [3]:
data=pd.read_excel("C:/Users/Dell/Pictures/sanchuop.xlsx")

In [4]:
data.head()

,Unnamed: 0,row_index,Name,content,Class,rating,final_sentiment,aspect,date,Channel,Type
0,0,1,Jasmine Jebarani,My cat s name is Hary She had to undergo hyst...,Quality of treatment,5,positive,"['undergo hysterectomy', 'successful doctors',...",2020-03-15,Google,Sanchu Hospital
1,1,2,SJC,Sanchu animal hospital is a high quality hospi...,Quality of treatment,4,positive,"['qualified team', 'papillion dog', 'good envi...",2020-04-14,Google,Sanchu Hospital
2,2,3,Krishnakumar M,Very well trained doctors Nice professional a...,Quality of treatment,5,positive,"['visit', 'pets', 'friends']",2020-03-15,Google,Sanchu Hospital
3,3,4,reshika mba,The Best hospital There are only very few avi...,Quality of treatment,5,positive,"['well thanks', 'mr annamalai', 'dr karunakara...",2020-03-15,Google,Sanchu Hospital
4,4,5,Manu Brahma,This is the best Animal hospital in city I vi...,Care,5,positive,"['visited times', 'pets like', 'vaccination', ...",2020-02-14,Google,Sanchu Hospital


In [5]:
data['text']=data['content']
#replacing the punctuations
data['text'] = data['text'].str.replace("[^a-zA-Z#]", " ")
#position to lowercase
data['text']=data['text'].str.lower()
#taking out nan
data = data[pd.notnull(data['text'])]

In [6]:
#lemmatization
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lemmatize_sentence)
#removing stopwords 
stop= set(stopwords.words('english')) 
data['text']=data['text'].apply(lambda x: [item for item in x if item not in stop])
#converting review column list to str
#ind['review']=ind['review'].apply(lambda x: ' '.join(map(str,x)))

In [ ]:
[x for x in data.text.sample(10)]

In [7]:
#topic modelling
#rule based topic modelling
#take out the all possible keywords coming under each class
#given those keywords into each list
#create class according to the data
Qualityoftreatment=['surgery','successfull','doctor','employee','treatment','quality','critical','condition','trained','professional',
                   'recommend','hospital','medical','equipment','treat','suffer','problem','animal','killer','effect','clinic','effective',
                    'examine','pain','injection','technically','experience','function','diagnose','medicine','pet','dog','injured','bleed',
                   'carelessness','collapse','high','dosage','well','maintain','locate','operation','operate','team','service',
                   'multispeciality','inject','treat','reaction','allergic','advice','emergency']

In [8]:
Hygiene=['facility','clean','environment','place','taxi','park','board','atmosphere','accommodative','maintenance']
Charges=['discount','charge','price','fees']
Care=['care','staff','kindness','help']

In [10]:
def matcher(query):
    s=0
    it=0
    wl=0
    c=0
         

    for x in query: 
        if x in Qualityoftreatment:        
            s +=1                    
        if x in Hygiene:
            it +=1                
        if x in Charges:
            wl +=1
        if x in Care:
            c +=1
        
    max_count=max(s,it,wl,c)
    print(max_count)
    if max_count==s:
        return 'Qualityoftreatment'
    elif max_count==it:
        return 'Hygiene'
    elif max_count==wl:
        return 'Charges'
    elif max_count==c:
        return 'Care'
data['class1']=data['text'].apply(matcher)


6
9
6
7
6
4
18
3
3
5
5
7
4
16
4
5
58
7
7
2
0
2
3
0
0
1
0
0


In [11]:
def matcher(query):
    s=0
    it=0
    wl=0
    c=0
         

    for x in query: 
        if x in Qualityoftreatment:        
            s +=1                    
        if x in Hygiene:
            it +=1                
        if x in Charges:
            wl +=1
        if x in Care:
            c +=1
        
    max_count=max(s,it,wl,c)
    max_count1=max_count-1
    if max_count1==s:
        return 'Qualityoftreatment'
    elif max_count1==it:
        return 'Hygiene'
    elif max_count1==wl:
        return 'Charges'
    elif max_count1==c:
        return 'Care'
data['class2']=data['text'].apply(matcher)


In [12]:
data.columns 

Index(['Unnamed: 0', 'row_index', 'Name', 'content', 'Class', 'rating',
       'final_sentiment', 'aspect', 'date', 'Channel', 'Type', 'text',
       'class1', 'class2'],
      dtype='object')

In [13]:
data=data.melt(id_vars =['Unnamed: 0', 'row_index', 'Name','content','Class','rating','date','final_sentiment','aspect',
            'text','Type','Channel'], value_vars =['class1','class2'],  
        var_name ='Variable_column', value_name ='classes') 
       
data = data[pd.notnull(data['classes'])]

data=data.drop(['Variable_column'],axis=1)

data.to_excel('C:/Users/Dell/Downloads/sanchuop.xlsx')

In [ ]:
import pandas as pd
one_year_review=pd.read_excel("C:/Users/Dell/Pictures/sanchuclassstr.xlsx")

In [ ]:
one_year_review.head()

In [ ]:
one_year_review['date']=one_year_review['date'].str.replace("months ago"," ").str.replace("month ago"," ") 

one_year_review['month'] = one_year_review['date'].str.strip() 

one_year_review['curr_date'] = pd.Timestamp('2020-05-14') 

one_year_review['curr_date'].dtype 

one_year_review['month_days'] = one_year_review['month'].astype(int)*30 

one_year_review['review_date'] = one_year_review['curr_date'] - pd.to_timedelta(one_year_review['month_days'],unit='D') 

In [ ]:
one_year_review.head()

In [ ]:
one_year_review=one_year_review[['row_index','Name','content','Class','rating','final_sentiment','aspect_adj_string','text','review_date']]

In [ ]:
one_year_review.head()

In [ ]:
one_year_review.to_excel("C:/Users/Dell/Pictures/sanchuop.xlsx")